In [22]:
# Python
import torch
import numpy as np
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from gram import GRAM
from dominant import DOMINANT
from conad import CONAD
from guide import GUIDE
from gcnae import GCNAE
from gaan import GAAN
from ocgnn import OCGNN
from gram_v2 import GNNVariant
from util import load_data
from sklearn.metrics import roc_auc_score, average_precision_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load data
name_dataset = 'PTC'
degree_as_tag = 'store_true'
graphs, num_classes = load_data(name_dataset, degree_as_tag)
node_feat_type = 1
if node_feat_type == 0:
    in_feats = 1

dataset = []
for graph in graphs:
    if node_feat_type:
        x = graph.node_features.clone().detach().to(torch.float)
        in_feats = graph.node_features.shape[1]
        adj_feats = graph.node_features.shape[0]
    else:
        x = torch.ones((graph.node_features.shape[0], in_feats), dtype=torch.float)
    edge_index = graph.edge_mat.clone().detach().T
    edge_index = torch.as_tensor(edge_index, dtype=torch.long).t().contiguous()
    y = torch.tensor([graph.label], dtype=torch.long)
    data = Data(x=x, edge_index=edge_index, y=y).to(device)
    dataset.append(data)

train_dataset = dataset[167:]
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = dataset[137:167]
test_loader = DataLoader(test_dataset, batch_size=1)

methods = {
    'gram': GRAM(),
    'dominant': DOMINANT(),
    'conad': CONAD(),
    'guide': GUIDE(),
    'gcnae': GCNAE(),
    'gaan': GAAN(),
    'ocgnn': OCGNN(),
    'gnn_variant': GNNVariant(
        in_channels=in_feats,
        hidden_channels=64,
        num_layers=3,
        out_channels=num_classes,
        dropout=0.5,
        gnn_type='gatv2'
    )
}

results = {}

for name, model in methods.items():
    print(f'Running {name}...')
    if name in ['gram', 'dominant', 'conad', 'gcnae', 'gaan', 'ocgnn']:
        model.fit(train_loader, in_feats)
    elif name == 'guide':
        model.fit(train_loader, in_feats, adj_feats)
    elif name == 'gnn_variant':
        model.fit(train_loader, device=device)
    else:
        continue

    score_pred = np.zeros(len(test_dataset))
    graph_label = np.zeros(len(test_dataset))
    t = 0
    for data in test_loader:
        y_label = 1 if data.y == 1 else 0
        data = data.to(device)
        if name == 'gram':
            scores = model.gradcam(data)
        else:
            scores = model.decision_function(data)
        score_pred[t] = sum(scores)
        graph_label[t] = y_label
        t += 1

    auc = roc_auc_score(graph_label, score_pred)
    ap = average_precision_score(graph_label, score_pred)
    results[name] = {'AUC': auc, 'AP': ap}
    print(f'{name}: AUC={auc:.4f}, AP={ap:.4f}')

print('Summary:')
for name, res in results.items():
    print(f'{name}: AUC={res["AUC"]:.4f}, AP={res["AP"]:.4f}')

loading data
# classes: 2
# maximum node tag: 4
# data: 344
Running gram...


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 35 but got size 9 for tensor number 1 in the list.